In [21]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pinxau1000/radioml2018")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2


In [22]:
!dir C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

 Volume in drive C is Windows SSD
 Volume Serial Number is 00BC-1571

 Directory of C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

06/03/2025  04:24 PM    <DIR>          .
06/03/2025  04:22 PM    <DIR>          ..
06/03/2025  04:24 PM               246 classes-fixed.json
06/03/2025  04:24 PM             1,632 classes-fixed.txt
06/03/2025  04:24 PM               262 classes.txt
06/03/2025  04:24 PM               172 datasets.desktop
06/03/2025  04:24 PM    21,449,148,312 GOLD_XYZ_OSC.0001_1024.hdf5
06/03/2025  04:24 PM            20,993 LICENSE.TXT
               6 File(s) 21,449,171,617 bytes
               2 Dir(s)  784,188,940,288 bytes free


In [23]:
dataset_path = "C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5"

In [24]:
!pip install torch seaborn

In [36]:
!pip install --upgrade torch

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 1.3/216.1 MB 13.4 MB/s eta 0:00:16
   - -------------------------------------- 6.3/216.1 MB 21.5 MB/s eta 0:00:10
   -- ------------------------------------- 12.1/216.1 MB 24.4 MB/s eta 0:00:09
   --- ------------------------------------ 18.4/216.1 MB 25.7 MB/s eta 0:00:08
   ---- ----------------------------------- 24.1/216.1 MB 26.3 MB/s eta 0:00:08
   ----- ---------------------------------- 28.3/216.1 MB 25.3 MB/s eta 0:00:08
   ------ --------------------------------- 33.6/216.1 MB 25.4 MB/s eta 0:00:08
   ------- -------------------------------- 39.3/216.1 MB 25.5 MB/s eta 0:00:07
   -------- ------------------------------- 45.1/216.1 MB 25.9 MB/s eta 0:00:07
   --------- ------------------------------ 50.9/216.1 MB 26.1 MB/s eta 0:00:07
   ---------- ----------------------------- 56.6/216.1 MB 26.5 MB/s eta 0:00:07
   ----------- ---------------------------- 62.4/21

  You can safely remove it manually.


1. Load dataset dan preprocessing

In [25]:
!pip install torchsummary

In [26]:
from torchsummary import summary
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [27]:
import h5py

# Misalkan kamu sudah memuat file HDF5
with h5py.File(dataset_path, 'r') as f:
    X = f['X'][:]  # Sinyal IQ
    Y = f['Y'][:]  # Label modulasi
    Z = f['Z'][:]  # Label SNR
    
    print(f"Shape of X (IQ data): {X.shape}")
    print(f"Shape of Y (Labels): {Y.shape}")
    print(f"Shape of Z (SNR): {Z.shape}")


Shape of X (IQ data): (2555904, 1024, 2)
Shape of Y (Labels): (2555904, 24)
Shape of Z (SNR): (2555904, 1)


2. Pendefinisian Class dan Objek

In [33]:
class BasicBlock(nn.Module):
     """ Basic Residual Block (used in ResNet) """
     def __init__(self, in_channels, out_channels):
         super(BasicBlock, self).__init__()
         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
         self.bn1 = nn.BatchNorm1d(out_channels)
         self.relu = nn.ReLU()
         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
         self.bn2 = nn.BatchNorm1d(out_channels)
         self.shortcut = nn.Sequential()
         if in_channels != out_channels:
             self.shortcut = nn.Sequential(
                 nn.Conv1d(in_channels, out_channels, kernel_size=1),
                 nn.BatchNorm1d(out_channels)
             )

     def forward(self, x):
         out = self.relu(self.bn1(self.conv1(x)))
         out = self.bn2(self.conv2(out))
         out += self.shortcut(x)  # residual connection
         out = self.relu(out)
         return out

class CNNResNetModel(nn.Module):
    def __init__(self, num_classes=8):
        super(CNNResNetModel, self).__init__()
        self.conv1 = nn.Conv1d(2, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        
        # ResNet blocks
        self.resnet_block1 = BasicBlock(64, 128)
        self.resnet_block2 = BasicBlock(128, 256)
        self.resnet_block3 = BasicBlock(256, 512)
        
        self.fc1 = nn.Linear(512 * 16, 1024)  # Adjust according to input length after pooling
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape from (batch_size, 1024, 2) to (batch_size, 2, 1024)
        # CNN part
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)  # shape becomes (batch_size, 64, 64)

        # ResNet part
        x = self.resnet_block1(x)  # (batch_size, 128, 64)
        x = self.pool(x)  # shape becomes (batch_size, 128, 32)
        
        x = self.resnet_block2(x)  # (batch_size, 256, 32)
        x = self.pool(x)  # shape becomes (batch_size, 256, 16)
        
        x = self.resnet_block3(x)  # (batch_size, 512, 16)
        x = self.pool(x)
        # Flattening the output for fully connected layers
        x = x.view(x.size(0), -1) # Flatten

        # Fully connected layers
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class RadioMLDataset(Dataset):
    """ Dataset for loading and preprocessing HDF5 file """
    def __init__(self, hdf5_path, classes):
        with h5py.File(hdf5_path, 'r') as f:
            self.X = f['X'][:]
            self.Y = f['Y'][:]
            self.SNR = f['Z'][:]
        self.classes = classes
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        # Normalize IQ samples to range [-1, 1]
        iq_data = self.X[idx]  # normalize
        label = self.Y[idx]
        if isinstance(label, np.ndarray):
            label = np.argmax(label)
        iq_data_normalized = iq_data / np.max(np.abs(iq_data))
        iq_tensor = torch.FloatTensor(iq_data_normalized)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return iq_tensor, label_tensor # torch.FloatTensor(iq_data), torch.tensor(label)

hdf5_path = dataset_path
classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']

dataset = RadioMLDataset(hdf5_path, classes)

# Melihat contoh data
print(f"Total samples: {len(dataset)}")
print(f"Shape of first sample (IQ): {dataset[0][0].shape}")
print(f"Label for first sample: {dataset[0][1]}")

Total samples: 2555904
Shape of first sample (IQ): torch.Size([1024, 2])
Label for first sample: 0


3. Subprogram Training dan Evaluasi

In [34]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return running_loss / total, correct / total, all_labels, all_preds

4. Program Utama untuk Training dan Evaluasi

In [35]:
def main():
    # Load HDF5 data
    hdf5_path = 'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5'  # Path to your HDF5 file
    classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']
    dataset = RadioMLDataset(hdf5_path, classes)

    # Split into train and test sets (80-20)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    # Model, loss function, optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNNResNetModel(num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Train the model
    num_epochs = 25
    for epoch in range(num_epochs):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader, criterion, device)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Final evaluation on the test set
    _, _, true_labels, predicted_labels = evaluate_model(model, test_loader, criterion, device)
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

if __name__ == '__main__':
    main()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x32768 and 8192x1024)

In [ ]:
'''# Jangan lupa tambahkan import ini di bagian atas file Anda jika belum ada
from torch.optim.lr_scheduler import ReduceLROnPlateau

def main():
    # Load HDF5 data
    # Pastikan path ini sudah benar sesuai dengan lokasi file Anda
    hdf5_path = 'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5'
    classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']
    dataset = RadioMLDataset(hdf5_path, classes)

    # Split into train and test sets (80-20)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    # Model, loss function, optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    model = CNNResNetModel(num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    
    # [MODIFIKASI] Menambahkan weight_decay (L2 Regularization) ke optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    
    # [MODIFIKASI] Menambahkan Learning Rate Scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, )

    # [MODIFIKASI] Menyiapkan variabel untuk menyimpan model terbaik dan riwayat training
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}
    best_test_acc = 0.0

    # Train the model
    num_epochs = 25
    for epoch in range(num_epochs):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader, criterion, device)

        # [MODIFIKASI] Menyimpan hasil training setiap epoch
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

        # [MODIFIKASI] Memanggil scheduler. Step dieksekusi berdasarkan akurasi testing.
        scheduler.step(test_acc)
        
        # [MODIFIKASI] Menyimpan model jika mendapatkan akurasi testing terbaik
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            torch.save(model.state_dict(), 'best_model_weights.pth')
            print(f"-> New best model saved with accuracy: {best_test_acc:.4f}")


    # [MODIFIKASI] Visualisasi Grafik Loss dan Akurasi
    print("\nTraining finished. Generating plots...")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['test_loss'], label='Test Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['test_acc'], label='Test Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()


    # Final evaluation on the test set using the best saved model
    print("\nLoading best model for final evaluation...")
    model.load_state_dict(torch.load('best_model_weights.pth'))
    
    _, _, true_labels, predicted_labels = evaluate_model(model, test_loader, criterion, device)
    cm = confusion_matrix(true_labels, predicted_labels)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title('Confusion Matrix of The Best Model')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # [MODIFIKASI] Menghitung dan menampilkan akurasi per kelas
    print("\nPer-Class Accuracy:")
    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    for i, acc in enumerate(class_accuracy):
        print(f"  - Accuracy for {classes[i]:<7}: {acc:.4f}")


if __name__ == '__main__':
    main()'''

Using device: cpu


RuntimeError: Given groups=1, weight of size [64, 2, 3], expected input[256, 1024, 2] to have 2 channels, but got 1024 channels instead